In [1]:
# for PyCharm
import os

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [2]:
import os

import pandas as pd
from pathlib import Path


from src.data.merger import Merger

In [3]:
datasets = Merger().get_formatted()
Merger.check_extra_symbols(datasets, column="title_without_color")

superkicks dict_keys(["'", '+', '"', '(', ')', '.', '#'])
sneakerbaas dict_keys(["'", '.', '(', ')', '+'])
footshop dict_keys(["'", '"', '.', '®', '*', '™', '+', '(', ')', '?', ',', ':'])
kickscrew dict_keys(['(', ')', "'", '.', '%', '+', '*', '"', '#', '!', ',', '~'])


In [4]:
datasets["superkicks"]

,brand,title,price,slug,brand_slug,collection_name,url,images_path,website,pricecurrency,title_without_color,color,title_merge
0,Adidas Originals,CAMPUS 00S 'CLOUD WHITE/BLUE',11999.0,campus-00s-cloud-whiteblue,adidas-originals,men-sneakers,https://www.superkicks.in/products/campus-00s-...,data/raw_new/images/superkicks/men-sneakers/ad...,superkicks,INR,CAMPUS 00S,CLOUD WHITE/BLUE,campus 00s
1,Converse,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE 'KH...,9999.0,wmns-run-star-motion-cx-platform-tortoise-khak...,converse,men-sneakers,https://www.superkicks.in/products/wmns-run-st...,data/raw_new/images/superkicks/men-sneakers/co...,superkicks,INR,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE,KHAKI/OFF WHITE,wmns run star motion cx platform tortoise
2,Nike,KEVIN DURANT KD16 EP 'BLACK/BLACK-DK SMOKE GRE...,13995.0,kevin-durant-kd16-ep-blackblackdk-smoke-greyco...,nike,men-sneakers,https://www.superkicks.in/products/kevin-duran...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,KEVIN DURANT KD16 EP,BLACK/BLACK-DK SMOKE GREY-COCONUT MILK,kevin durant kd16 ep
3,Puma,STAPLE B SUEDE 'DARK CHOCOLATE-RHUBARB',9999.0,staple-b-suede-dark-chocolaterhubarb,puma,men-sneakers,https://www.superkicks.in/products/staple-b-su...,data/raw_new/images/superkicks/men-sneakers/pu...,superkicks,INR,STAPLE B SUEDE,DARK CHOCOLATE-RHUBARB,staple b suede
4,Nike,DUNK LOW RETRO PREMIUM 'CACAO WOW/COMET BLUE-M...,9695.0,dunk-low-retro-premium-cacao-wowcomet-bluemars...,nike,men-sneakers,https://www.superkicks.in/products/dunk-low-re...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,DUNK LOW RETRO PREMIUM,CACAO WOW/COMET BLUE-MARS STONE-PHANTOM,dunk low retro premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,Nike,WMN'S DUNK LOW PREMIUM MF 'RUSH FUCHSIA/RUSH F...,10795.0,wmns-dunk-low-premium-mf-rush-fuchsiarush-fuch...,nike,women-skateboard-sneakers,https://www.superkicks.in/products/wmns-dunk-l...,data/raw_new/images/superkicks/women-skateboar...,superkicks,INR,WMN'S DUNK LOW PREMIUM MF,RUSH FUCHSIA/RUSH FUCHSIA-RUSH FUCHSIA,wmns dunk low premium mf
1086,Converse,RUN STAR MOTION BLACK,8999.0,run-star-motion-black,converse,women-skateboard-sneakers,https://www.superkicks.in/products/run-star-mo...,data/raw_new/images/superkicks/women-skateboar...,superkicks,INR,RUN STAR MOTION BLACK,,run star motion black
1087,Converse,RUN STAR HIKE LOW 'WHITE',5999.0,run-star-hike-low-white,converse,women-skateboard-sneakers,https://www.superkicks.in/products/run-star-hi...,data/raw_new/images/superkicks/women-skateboar...,superkicks,INR,RUN STAR HIKE LOW,WHITE,run star hike low
1088,Nike,WMN'S DUNK HIGH 'BLACK/WHITE',9295.0,wmns-dunk-high-blackwhite,nike,women-skateboard-sneakers,https://www.superkicks.in/products/wmns-dunk-h...,data/raw_new/images/superkicks/women-skateboar...,superkicks,INR,WMN'S DUNK HIGH,BLACK/WHITE,wmns dunk high


In [5]:
# apply deletion of colors

In [6]:
for key, value in datasets.items():
    print(value.shape)

(1090, 13)
(999, 14)
(4852, 13)
(4900, 10)


In [7]:
# same columns
same_columns = set.intersection(*[set(datasets[key].columns) for key in datasets])
same_columns
# no colors and collection-name

{'brand',
 'color',
 'images_path',
 'price',
 'pricecurrency',
 'title',
 'title_merge',
 'title_without_color',
 'url',
 'website'}

In [8]:
merged_datasets = pd.concat(list(datasets.values()), ignore_index=True)
merged_datasets

,brand,title,price,slug,brand_slug,collection_name,url,images_path,website,pricecurrency,title_without_color,color,title_merge,description_color
0,Adidas Originals,CAMPUS 00S 'CLOUD WHITE/BLUE',11999.0,campus-00s-cloud-whiteblue,adidas-originals,men-sneakers,https://www.superkicks.in/products/campus-00s-...,data/raw_new/images/superkicks/men-sneakers/ad...,superkicks,INR,CAMPUS 00S,CLOUD WHITE/BLUE,campus 00s,NaN
1,Converse,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE 'KH...,9999.0,wmns-run-star-motion-cx-platform-tortoise-khak...,converse,men-sneakers,https://www.superkicks.in/products/wmns-run-st...,data/raw_new/images/superkicks/men-sneakers/co...,superkicks,INR,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE,KHAKI/OFF WHITE,wmns run star motion cx platform tortoise,NaN
2,Nike,KEVIN DURANT KD16 EP 'BLACK/BLACK-DK SMOKE GRE...,13995.0,kevin-durant-kd16-ep-blackblackdk-smoke-greyco...,nike,men-sneakers,https://www.superkicks.in/products/kevin-duran...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,KEVIN DURANT KD16 EP,BLACK/BLACK-DK SMOKE GREY-COCONUT MILK,kevin durant kd16 ep,NaN
3,Puma,STAPLE B SUEDE 'DARK CHOCOLATE-RHUBARB',9999.0,staple-b-suede-dark-chocolaterhubarb,puma,men-sneakers,https://www.superkicks.in/products/staple-b-su...,data/raw_new/images/superkicks/men-sneakers/pu...,superkicks,INR,STAPLE B SUEDE,DARK CHOCOLATE-RHUBARB,staple b suede,NaN
4,Nike,DUNK LOW RETRO PREMIUM 'CACAO WOW/COMET BLUE-M...,9695.0,dunk-low-retro-premium-cacao-wowcomet-bluemars...,nike,men-sneakers,https://www.superkicks.in/products/dunk-low-re...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,DUNK LOW RETRO PREMIUM,CACAO WOW/COMET BLUE-MARS STONE-PHANTOM,dunk low retro premium,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11836,adidas,Adidas Ultraboost 1.0 Shoes 'Valentines Day',123.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-ultr...,[data/raw/images/kickscrew/adidas/adidas-ultra...,kickscrew,USD,Adidas Ultraboost 1.0 Shoes,Valentines Day,adidas ultraboost 10 shoes,NaN
11837,adidas,Adidas Originals Ozweego Shoes 'Magic Beige Cl...,102.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Ozweego Shoes,Magic Beige Clay Strata,adidas originals ozweego shoes,NaN
11838,adidas,Adidas Originals Adilette Slides 'Off White Br...,56.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Adilette Slides,Off White Bright Orange,adidas originals adilette slides,NaN
11839,adidas,Adidas Originals Forum Low Shoes 'White Grey Red',84.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Forum Low Shoes,White Grey Red,adidas originals forum low shoes,NaN


In [9]:
# group by по названиям, если все названия будут одинаковые, то все дублирующие кроссы сложатся в одно.
test = merged_datasets.groupby("title_merge").agg({
    "brand": list,
    "collection_name": list,
    "color": list,
    "images_path": list,
    "price": list,
    "pricecurrency": list,
    "url": list,
    "website": list
}).reset_index().sort_values(by="title_merge")
test

,title_merge,brand,collection_name,color,images_path,price,pricecurrency,url,website
0,01 low,"[Autry, Autry]","[category-women, category-men]","[White, Mustard]",[data/raw_new/images/sneakerbaas/category-wome...,"[129.99, 149.99]","[EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas]"
1,01 low m,"[Autry, Autry, Autry]","[category-men, category-men, category-men]","[BLACK, BI VIOLET, BI EDEN]",[data/raw_new/images/sneakerbaas/category-men/...,"[149.99, 149.99, 149.99]","[EUR, EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas, sneakerbaas]"
2,01 low w,"[Autry, Autry, Autry, Autry]","[category-women, category-women, category-wome...","[BI POW, WHT/CORAL, SALTWATER, BI IVORY]",[data/raw_new/images/sneakerbaas/category-wome...,"[149.99, 149.99, 149.99, 149.99]","[EUR, EUR, EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas, sneakerbaas, sneake..."
3,01 low wom,[Autry],[category-women],[WHT/PETROL],[data/raw_new/images/sneakerbaas/category-wome...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
4,180 tones,"[PUMA Sportstyle, PUMA Sportstyle]","[category-unisex, category-unisex]","[Glacial Gray, Toasted Almond]",[data/raw_new/images/sneakerbaas/category-unis...,"[95.99, 95.99]","[EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas]"
...,...,...,...,...,...,...,...,...,...
5224,zoom verona slip,"[Nike Skateboarding, Nike Skateboarding]","[category-men, category-men]","[Leo Baker, Light Dew]",[data/raw_new/images/sneakerbaas/category-men/...,"[49.99, 49.99]","[EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas]"
5225,zoom verona slip x leticia bufon,[Nike Skateboarding],[category-unisex],[],[data/raw_new/images/sneakerbaas/category-unis...,[39.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
5226,zoom vomero 5 sp,"[Nike, Nike]","[men-sneakers, men-sneakers]","[VAST GREY/VAST GREY-BLACK-SAIL, ANTHRACITE/AN...",[data/raw_new/images/superkicks/men-sneakers/n...,"[14995.0, 14995.0]","[INR, INR]",[https://www.superkicks.in/products/zoom-vomer...,"[superkicks, superkicks]"
5227,zx 22 boost,[Adidas Originals],[category-men],[Cream White],[data/raw_new/images/sneakerbaas/category-men/...,[59.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]


In [13]:
test.to_csv("data/raw/metadata/dataset.csv", index=False)

In [10]:
# sizes
print(merged_datasets.shape)
print(test.shape)

(11841, 14)
(5229, 9)


In [11]:
test[["title_merge", "website"]].to_csv("temp.csv", index=False)

In [12]:
titles = test["title"].tolist()

KeyError: 'title'

In [12]:
!pip install thefuzz -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
from thefuzz import fuzz

def similarity(string1, string2):
    return fuzz.token_sort_ratio(string1, string2)

title = "vans - ua og authentic lx yellow"

similarities = []
for text in titles:
    similarities.append((text, similarity(title, text)))

similarities.sort(key=lambda x: x[1])
similarities[-10:]

[('vans ua og authentic lx vault checkerboard', 69),
 ('vans vault authentic one piece vlt lx', 69),
 ('vans vault og authentic frayed lx', 70),
 ('vans vault og authentic lx canvas', 70),
 ('authentic los vans', 71),
 ('vans og authentic lx stressed', 71),
 ('vans ua authentic vr3 pw lx', 77),
 ('vans ua og authentic lx suede', 78),
 ('vans vault og authentic lx', 79),
 ('vans og authentic lx', 80)]

In [18]:
# эта тема лучше всего находит похожие

In [14]:
test.to_csv("temp.csv")